# Deep-LDA - Alanine dipeptide

Tutorial for the training of the Deep-LDA collective variable, using the Alanine Dipeptide system as example with the *interatomic distances* as input descriptors.

Reference: Bonati, Rizzi and Parrinello, J. Phys. Chem. Lett., 11, 2998-3004 (2020).

This page is dapted from [MLCVS docs](https://mlcvs.readthedocs.io/en/latest/notebooks/ala2_deeplda.html)

Download data: `https://github.com/luigibonati/mlcvs/tree/main/docs/notebooks/data'

In [2]:
import os
from pathlib import Path
dir_nb = Path(globals()['_dh'][0])

import torch
import numpy as np
import matplotlib.pyplot as plt

# dir_base = dir_nb/'deep_LDA'
# if not dir_base.is_dir(): dir_base.mkdir()
# os.chdir(dir_base)

Plotting functions

In [3]:
def plot_ramachandran(x,y,z,scatter=None,ax=None):
    # Setup plot
    if ax is None:
        _, ax = plt.subplots(figsize=(5,4.), dpi=100)
        ax.set_title(f'Ramachandran plot')

    # Plot countour plot
    h = ax.hexbin(x,y,C=z,cmap='fessa')
    cbar = plt.colorbar(h,ax=ax)
    cbar.set_label(f'Deep-LDA CV')

    axs[0].set_xlabel(r'$\phi$ [rad]')
    axs[0].set_ylabel(r'$\psi$ [rad]')

def plot_cv_histogram(s,label=None,ax=None,**kwargs):
    # Setup plot
    if ax is None:
        _, ax = plt.subplots(figsize=(5,4.), dpi=100)
        ax.set_title('Histogram')

    if (type(s)==torch.Tensor):
        s = s.squeeze(1).detach().numpy()

    # Plot histogram
    ax.hist(s,**kwargs)
    if label is not None:
        ax.set_xlabel(label)



## Load data

Load the descriptors from PLUMED COLVAR files (one unbiased run for every metastable state).

In [14]:
from mlcvs.utils.io import load_dataframe

filenames = [ "data/ala2_md/COLVAR_stateA", "data/ala2_md/COLVAR_stateB" ]

X, y = [], []

for i,file in enumerate(filenames):
    data = load_dataframe(file)[::4]

    # Descriptors
    selection = 'd'
    X.append( data.filter(regex=selection).values )
    names = data.filter(regex=selection).columns.values

    # Labels
    y.append( np.full(len(data),i) )

X = torch.Tensor( np.vstack(X) )
y = torch.Tensor( np.hstack(y) )

n_features = X.shape[1]

Create a dataset, split in train and validation, and initialize a `FastTensorDataLoader’ for efficient training.

In [21]:
from torch.utils.data import TensorDataset,random_split
from mlcvs.utils.data import FastTensorDataLoader

dataset = TensorDataset(X,y)
train_size = int(0.9 * len(dataset))
valid_size = len(dataset) - train_size

train_data, valid_data = random_split(dataset,[train_size,valid_size])
train_loader = FastTensorDataLoader(train_data)
valid_loader = FastTensorDataLoader(valid_data)

## Train CV

Inizialize the neural network and the optimizer and define when to stop the training (EarlyStopping or after a given number of epochs).

| Parameter | Type | Description |
| :- | :- | :- |
| **Neural network** |
| nodes | list | NN architecture (last value equal to the number of hidden layers which are input of LDA) |
| activ_type | string | Activation function (relu,tanh,elu,linear) |
| n_eig | int | Number of eigenvalues to optimize (or if loss_type=single which one to select) |
| **Optimization** |
| lrate | float | Learning rate |
| sw_reg | float | S_w matrix regularization | 
| l2_reg | float | L2 regularization |
| num_epochs | int | Number of epochs |
| **Early Stopping** |
| earlystop | bool | Whether to use early stopping based on validation loss |
| es_patience | int | Number of epochs before stopping |
| es_consecutive | bool | Whether es_patience should count consecutive (True) or cumulative patience |
| es_min_delta | float | Minimum decrease of validation loss |
| **Log** |
| log_every | int | How often print the train/valid loss during training |


In [24]:
from mlcvs.lda import DeepLDA_CV

#------------- PARAMETERS -------------
nodes             = [n_features,30,30,5]

lrate             = 0.001
sw_reg            = 0.05
l2_reg            = 1e-5

num_epochs        = 1000
earlystop         = True
es_patience       = 20
es_consecutive    = True
es_min_delta      = 0.02

log_every         = 100
#--------------------------------------

# DEVICE
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# MODEL
model = DeepLDA_CV(nodes)
model.set_device(device)

# OPTIMIZER
opt = torch.optim.Adam(model.parameters(), lr=lrate, weight_decay=l2_reg)
model.set_optimizer(opt)

# REGULARIZATION
model.set_regularization(sw_reg=sw_reg)
model.set_earlystopping(patience=es_patience,consecutive=es_consecutive,min_delta=es_min_delta)

# TRAIN
model.fit(train_loader, valid_loader, standardize_inputs = True, log_every=log_every)

# standardize outputs
#model.standardize_outputs(train_data[0].to(device))

[Warning] Normalization: the following features have a range of values < 1e-6: tensor([[ 0],
        [ 9],
        [10],
        [24],
        [30],
        [31],
        [39],
        [40],
        [44]])


/home/tha/app/miniconda3/envs/py39mlcvs/lib/python3.9/site-packages/mlcvs/lda/lda.py:118: UserWarning: torch.cholesky is deprecated in favor of torch.linalg.cholesky and will be removed in a future PyTorch release.
L = torch.cholesky(A)
should be replaced with
L = torch.linalg.cholesky(A)
and
U = torch.cholesky(A, upper=True)
should be replaced with
U = torch.linalg.cholesky(A).mH().
This transform will produce equivalent results for all valid (symmetric positive definite) inputs. (Triggered internally at /opt/conda/conda-bld/pytorch_1670525495809/work/aten/src/ATen/native/BatchLinearAlgebra.cpp:1615.)
  L = torch.cholesky(S_w, upper=False)
/home/tha/app/miniconda3/envs/py39mlcvs/lib/python3.9/site-packages/mlcvs/lda/lda.py:127: UserWarning: torch.symeig is deprecated in favor of torch.linalg.eigh and will be removed in a future PyTorch release.
The default behavior has changed from using the upper triangular portion of the matrix by default to using the lower triangular portion.
L, _ 

epoch           loss_train      loss_valid      
100             -40.439 -40.343 
200             -51.941 -52.373 
300             -62.380 -62.299 
INFO: Early stopping
396             -62.579 -62.494 


In [25]:
# get back to CPU
model.to('cpu')

DeepLDA_CV(
  (nn): Sequential(
    (0): Linear(in_features=45, out_features=30, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=30, out_features=30, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=30, out_features=5, bias=True)
  )
)